.

- 참고자료 : https://www.youtube.com/watch?v=vS1L11qnoS8


#### 1. 개요

아래 그림과 같이 프라이빗 서브넷에 EMR을 생성하고, 퍼블릭 서브넷의 베스천 서버의 터널링을 이용하여 서브넷 내에 EMR 웹 인터페이스를 접근하는 아키텍처 구현

![2](https://user-images.githubusercontent.com/41605276/82114682-3bf62900-9799-11ea-960e-192ad0de9742.jpg)

#### 2. 구현순서

STEP 1) 네트워크 환경구축

1-1) VPC 생성

1-2) 서브넷 생성

1-3) 인터넷 게이트웨이 생성

1-4) NAT 게이트웨이 생성

1-5) 라우팅 테이블 생성 및 설정

STEP 2) EMR 생성

2-1) EMR-s3 엔드포인트 생성 및 설정

2-2) EMR 생성

STEP 3) 베스천 서버 생성

SETP 4) 터널링 적용

4-1) 터널링 작업

4-2) 팍시프록시 설정

STEP 5) 구현결과 확인

#### STEP 1) 네트워크 환경구축

아래 그림과 같은 형태의 네트워크를 먼저 구축한다.(아래 그림과 아이피는 상이하게 구현하였음)

![3](https://user-images.githubusercontent.com/41605276/82113663-a9528b80-9792-11ea-8797-47b281252d97.PNG)

1-1) VPC 생성

![4](https://user-images.githubusercontent.com/41605276/82113593-25989f00-9792-11ea-92b1-57b6cacf8bd2.PNG)

1-2) 서브넷 생성

public 서브넷, private 서브넷 순차적으로 생성

![5](https://user-images.githubusercontent.com/41605276/82113636-86c07280-9792-11ea-88e3-03e01cd52788.png)

1-3) 인터넷 게이트웨이 생성

![6](https://user-images.githubusercontent.com/41605276/82113691-ed459080-9792-11ea-9bdf-f0b36dbc4590.PNG)

1-4) NAT 게이트웨이 생성

![7](https://user-images.githubusercontent.com/41605276/82113720-1f56f280-9793-11ea-91fe-b0a778753230.PNG)

1-5) 라우팅 테이블 생성 및 설정

![8](https://user-images.githubusercontent.com/41605276/82113785-9db39480-9793-11ea-8c24-ac3cce1fd25c.png)

#### STEP 2) EMR 생성

아래 그림과 같은 형태로 프라이빗 서브넷 내에 EMR을 구성한다.

![1](https://user-images.githubusercontent.com/41605276/82113824-dbb0b880-9793-11ea-9afc-5be021ecf1e0.PNG)

2-1) EMR-s3 엔드포인트 생성 및 프라이빗 서브넷 라우팅 테이블 설정

![9](https://user-images.githubusercontent.com/41605276/82113899-3e09b900-9794-11ea-849d-373cb2067295.png)

2-2) EMR 생성

먼저 아래와 같이 베스천에서 사용할 보안그룹을 생성해준다. ssh 포트를 열어준다.

![11](https://user-images.githubusercontent.com/41605276/82114086-7a89e480-9795-11ea-8632-c712f5b3e7cd.png)

그 다음에 default 옵션, default 보안그룹으로 EMR을 생성하고, 마스터 노드의 보안그룹에서 '베스천에서 사용할 보안그룹'을 추가해준다.

EMR 메뉴에서 'VPC 서브넷' 메뉴로 들어사거 NAT게이트웨이와 S3 endpoint가 설정이 되어 있는지 확인한다.

![10](https://user-images.githubusercontent.com/41605276/82114005-ec156300-9794-11ea-8be5-c02eb67d559e.png)

#### STEP 3) 베스천 서버 생성

퍼블릭 서브넷에 베스천 서버를 생성한다. 참고로 keypair는 EMR에서 사용하는 keypair와 동일한 것으로 사용했다.

![12](https://user-images.githubusercontent.com/41605276/82114139-d05e8c80-9795-11ea-9fff-dae790058826.png)

#### SETP 4) 터널링 적용

4-1) 터널링 작업

먼저 베스천 서버에 들어가서 keypair를 특정 폴더에 업로드 시킨다. (사이버덕을 사용하였음)

그 다음에 아래와 같이 local에서 쉘 창하나를 띄워서 bastion 접속 -> bastion에서 업로드한 keypair를 이용하여 EMR 마스터노드 접속해서 pyspark 같은 어플리케이션 하나를 실행해본다. 그리고 이 쉘창은 계속 띄워놓는다.

만약에  bastion에서 업로드한 keypair를 이용하여 EMR 마스터노드 접속할때 keypair 퍼미션 문제가 발생하면 아래와 같이 조치해준다.

![13](https://user-images.githubusercontent.com/41605276/82114307-f7698e00-9796-11ea-9d20-e6b8ced85141.png)

4-2) 팍시프록시 설정

아래 그림과 같이 크롬에 팍시프록시를 다운받고, 적용해준다.

** 팍시 프록시 다운로드 url : https://chrome.google.com/webstore/detail/foxyproxy-standard/gcknhkkoolaabfmlnjonogaaifnjlfnp?hl=en

![14](https://user-images.githubusercontent.com/41605276/82114375-75c63000-9797-11ea-8914-f47c520f3e89.png)

아래 코드는 위에 그림에서 사용한 `foxyproxy-settings.xml` 이다. EMR 콘솔에서도 이 내용을 확인할 수 있다.

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>
<foxyproxy>
    <proxies>
        <proxy name="emr-socks-proxy" id="2322596116" notes="" fromSubscription="false" enabled="true" mode="manual" selectedTabIndex="2" lastresort="false" animatedIcons="true" includeInCycle="true" color="#0055E5" proxyDNS="true" noInternalIPs="false" autoconfMode="pac" clearCacheBeforeUse="false" disableCache="false" clearCookiesBeforeUse="false" rejectCookies="false">
            <matches>
                <match enabled="true" name="*ec2*.amazonaws.com*" pattern="*ec2*.amazonaws.com*" isRegEx="false" isBlackList="false" isMultiLine="false" caseSensitive="false" fromSubscription="false" />
                <match enabled="true" name="*ec2*.compute*" pattern="*ec2*.compute*" isRegEx="false" isBlackList="false" isMultiLine="false" caseSensitive="false" fromSubscription="false" />
                <match enabled="true" name="10.*" pattern="http://10.*" isRegEx="false" isBlackList="false" isMultiLine="false" caseSensitive="false" fromSubscription="false" />
                <match enabled="true" name="*10*.amazonaws.com*" pattern="*10*.amazonaws.com*" isRegEx="false" isBlackList="false" isMultiLine="false" caseSensitive="false" fromSubscription="false" />
                <match enabled="true" name="*10*.compute*" pattern="*10*.compute*" isRegEx="false" isBlackList="false" isMultiLine="false" caseSensitive="false" fromSubscription="false" />
                <match enabled="true" name="*.compute.internal*" pattern="*.compute.internal*" isRegEx="false" isBlackList="false" isMultiLine="false" caseSensitive="false" fromSubscription="false" />
                <match enabled="true" name="*.ec2.internal*" pattern="*.ec2.internal*" isRegEx="false" isBlackList="false" isMultiLine="false" caseSensitive="false" fromSubscription="false" />
            </matches>
            <manualconf host="localhost" port="8157" socksversion="5" isSocks="true" username="" password="" domain="" />
        </proxy>
    </proxies>
</foxyproxy>

#### STEP 5) 구현결과 확인

아래와 같이 쉘창을 하나 띄워서 `ssh -i [keypair 이름].pem -ND 8157 ec2-user@[bastion public ip]` 명령어를 실행한다. 그러면 특별히 뭐가 출력이 되는 상태가 아닌 그 명렁어가 계속 실행되는 상태로 유지가 될것이다. 그런 다음에 크롬으로가서 `[EMR master의 private ip]/8888`로 접속하면 아래 그림과 같이 EMR의 Hue로 잘 접속이 되는 것을 확인 할 수 있다. EMR 콘솔에서도 새로고침 하면 HUE나 다른 WEB UI 인터페이스 주소가 활성화되어 클릭하면 접속할 수 있을것이다.

![15](https://user-images.githubusercontent.com/41605276/82114551-60053a80-9798-11ea-9d6c-a7f41803e5d7.PNG)


** 위와 같은 방식을 다이나믹 포트 포워딩이라고 한다.

참고로 keypair가 없고, 계정으로 베스천서버에 접속하는 방식이라면 로컬 쉘에서 명령어를 실행할때 `ssh -i [keypair 이름].pem -ND 8157 ec2-user@[bastion public ip]` 대신에 아래와 같은 명령어를 사용하면 된다.

![22](https://user-images.githubusercontent.com/41605276/82959722-6181fe80-9ff3-11ea-96bd-8ccb9573ba91.png)

In [ ]:
ssh -ND 8157 [베스천서버에서 나의 계정이름]@[베스천서버의 퍼블릭 아이피]